In [6]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

data_path = "../../datasets"

if torch.accelerator.is_available():
    device = torch.accelerator.current_accelerator()
    print(f"Using accelerator '{device}'")

    if device.type == "cuda":
        torch.backends.cudnn.deterministic = True
else:
    device = torch.device("cpu")
    print("WARN: No accelerator found, running on CPU")


transform = transforms.Compose(
    [
        transforms.ToTensor(),
        # normalize by mean and standard deviation, such that dataset has mean of 0.5 and standard deviation of 1
        transforms.Normalize((0.1307,), (0.3081,)),
    ]
)

train_dataset = datasets.MNIST(
    data_path,
    train=True,
    download=True,
    transform=transform,
)

test_loader = DataLoader(
    datasets.MNIST(data_path, train=False, download=False, transform=transform),
    shuffle=False,
    drop_last=False,
    batch_size=10000,
    generator=torch.Generator(),
)

Using accelerator 'mps'


In [7]:
class MnistCnn(torch.nn.Module):
    def __init__(self):
        super(MnistCnn, self).__init__()

        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv2 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.dropout1 = torch.nn.Dropout(p=0.25)
        self.dropout2 = torch.nn.Dropout(p=0.5)
        self.fc1 = torch.nn.Linear(in_features=9216, out_features=128)
        self.fc2 = torch.nn.Linear(in_features=128, out_features=10)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.nn.functional.relu(x)
        x = self.conv2(x)
        x = torch.nn.functional.relu(x)
        x = torch.nn.functional.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = torch.nn.functional.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)

        # Log softmax rather than softmax due to negative log likelihood loss.
        # log_softmax rather than two separate operations for numerical stability
        output = torch.nn.functional.log_softmax(x, dim=1)

        return output

In [8]:
from components.data_splitting import index_uniformly, partition_dataset
from fedavg.server import FedAvgServer

client_datasets = partition_dataset(
    train_dataset,
    index_uniformly(train_dataset, partitions_count=100, generator_or_seed=42),
)

torch.manual_seed(42)
fedavg_server = FedAvgServer(
    device=device,
    model=MnistCnn().to(device),
    learning_rate=0.01,
    batch_size=200,
    client_subsets=client_datasets,
    client_fraction=0.1,
    local_epochs=1,
    seed=42,
)

result_fedavg = fedavg_server.run(rounds=10, test_loader=test_loader)

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

clients:   0%|          | 0/10 [00:00<?, ?it/s]

clients:   0%|          | 0/10 [00:00<?, ?it/s]

clients:   0%|          | 0/10 [00:00<?, ?it/s]

clients:   0%|          | 0/10 [00:00<?, ?it/s]

...

clients:   0%|          | 0/10 [00:00<?, ?it/s]

clients:   0%|          | 0/10 [00:00<?, ?it/s]

clients:   0%|          | 0/10 [00:00<?, ?it/s]

clients:   0%|          | 0/10 [00:00<?, ?it/s]

clients:   0%|          | 0/10 [00:00<?, ?it/s]

In [9]:
result_fedavg.as_df()

,round,algorithm,clients_count,active_clients_fraction,batch_size,local_epochs_count,learning_rate,seed,wall_time,message_count,test_accuracy
0,1,FedAvg,100,0.1,200,1,0.01,42,1.080764,20,0.0937
1,2,FedAvg,100,0.1,200,1,0.01,42,0.834178,40,0.1038
2,3,FedAvg,100,0.1,200,1,0.01,42,0.835917,60,0.1263
3,4,FedAvg,100,0.1,200,1,0.01,42,0.895523,80,0.2450
4,5,FedAvg,100,0.1,200,1,0.01,42,0.834426,100,0.3585
5,6,FedAvg,100,0.1,200,1,0.01,42,0.838036,120,0.4036
6,7,FedAvg,100,0.1,200,1,0.01,42,0.838551,140,0.4502
7,8,FedAvg,100,0.1,200,1,0.01,42,0.862022,160,0.5202
8,9,FedAvg,100,0.1,200,1,0.01,42,0.824295,180,0.5883
9,10,FedAvg,100,0.1,200,1,0.01,42,0.830334,200,0.6386
